In [1]:
# Data Managment
import sqlite3
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions

# Text Managment
import re
from sentence_transformers import SentenceTransformer, util
from getpass import getpass
import tiktoken

from openai import OpenAI

# Miscellanium
import os

/home/fernando/Projectos/metrocuadrado/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
con = sqlite3.connect('../data/database.db')
cur = con.cursor()

In [3]:
query = """
SELECT 
    *
FROM
    Properties
"""
df = pd.read_sql_query(query, con)
df.fillna(0, inplace=True)
df.head()

,id_property,id_offeror,businessType,comments,url,propertyType,builtArea,area,bathroomsNumber,roomsNumber,parkingNumber,price,stratum,lat,lon
0,164-M4623204,3381,Arriendo,Bellisimo aparta estudio super ubicadisimo a u...,https://www.metrocuadrado.com/inmueble/arriend...,Apartamento,28.00,28.00,1.0,1.0,1.0,1600000.0,4,4.653850,-74.056250
1,164-M4617624,3381,Arriendo,"Apartamento En Cabrera, al frente del parque ...",https://www.metrocuadrado.com/inmueble/arriend...,Apartamento,233.00,235.00,3.0,3.0,2.0,9000000.0,6,4.667340,-74.049110
2,3424-M2095254,2920,Venta,"CASALOTE MEDIANERO DE DOS PISOS MAS TERRAZA, E...",https://www.metrocuadrado.com/inmueble/venta-l...,Lote o Casalote,188.00,187.50,0.0,0.0,0.0,360000000.0,2,4.563945,-74.132912
3,10858-M4680630,5015,Venta,RentAHouse Colombia vende apartamento en villa...,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento,74.72,74.72,3.0,3.0,2.0,380000000.0,4,4.744681,-74.032646
4,10858-M4680631,5015,Venta,Excelente apartamento en venta y arriendo ubic...,https://www.metrocuadrado.com/inmueble/venta-a...,Apartamento,66.61,66.61,2.0,2.0,2.0,470000000.0,5,4.694066,-74.060853


In [4]:
df = df.reset_index()
df = df.rename(columns = {'index': 'ids'}).astype({'ids':str})

In [5]:
df['text'] = df.apply(lambda x: (x['comments']+' '+ x['businessType']+' '+ x['propertyType']).strip(), axis=1)

In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/home/fernando/Projectos/metrocuadrado/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
embeddings = model.encode(sentences=df['text'], batch_size=64, show_progress_bar=True)

Batches: 100%|██████████| 130/130 [00:15<00:00,  8.41it/s]


In [8]:
df['embeddings'] = embeddings.tolist()

In [9]:
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

In [ ]:
# openai_ef = embeddings_functions.OpenAIEmbeddingFunction(
#     api_key = 'xxxxxx',
#     model_name = 'embedding-ada-002'
# )

In [10]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path = '../data/properties_embedding_db')

In [11]:
db = client_persistent.create_collection(name = 'properties_embedding_db', embedding_function=sentence_transformer_ef)

In [12]:
db.add(
    ids=df['ids'].tolist(),
    embeddings=df['embeddings'].tolist(),
    metadatas= df.drop(['ids', 'embeddings', 'text'],axis=1).to_dict('records')
)

In [ ]:
def search(query, businessType, propertyType, stratum, top_k=3):
    db.query(
        query_texts= [query],
        n_results=top_k
    )

    if stratum:
        pass
    else:
        stratum = 0
    


In [32]:
def search(query, top_k=5):
    results = db.query(
            query_texts= [query],
            n_results=top_k
    )
    
    return pd.DataFrame(results['metadatas'][0])

search('D1 por ahí')

,area,bathroomsNumber,builtArea,businessType,comments,id_offeror,id_property,lat,lon,parkingNumber,price,propertyType,roomsNumber,stratum,url
0,33.55,1.0,38.85,Venta,"Apartamento con 2 habitaciones, 1 baño, sala c...",0,MC4638366,4.736210,-74.085260,0.0,185000000.0,Apartamento,2.0,3,https://www.metrocuadrado.com/inmueble/venta-a...
1,30.00,1.0,30.00,Arriendo,Hermoso apartamento con solo 1 año de uso. Cue...,0,MC4749686,4.757689,-74.050810,0.0,1350000.0,Apartamento,1.0,4,https://www.metrocuadrado.com/inmueble/arriend...
2,40.00,1.0,42.00,Arriendo,Apartamento nuevo ubicado en un sexto piso ext...,0,MC4747358,4.757783,-74.052734,0.0,1400000.0,Apartamento,1.0,3,https://www.metrocuadrado.com/inmueble/arriend...
3,250.00,5.0,250.00,Arriendo,Este precioso apartamento está en un séptimo p...,4420,562-M4661403,4.622794,-74.090960,3.0,11000000.0,Apartamento,4.0,6,https://www.metrocuadrado.com/inmueble/arriend...
4,102.00,3.0,106.00,Venta,"Consta de: sala comedor, cocina integral, con ...",0,MC4690550,4.752923,-74.037380,2.0,535000000.0,Apartamento,3.0,4,https://www.metrocuadrado.com/inmueble/venta-a...
